#LOAD DATA

In [ ]:
pip install kagglehub


In [ ]:
import kagglehub

# Tải dataset VGGFace2 từ Kaggle
dataset_path = kagglehub.dataset_download("hearfool/vggface2")

print("Dataset được tải về tại:", dataset_path)


100%|██████████| 2.32G/2.32G [01:23<00:00, 29.7MB/s]

Extracting files...


Dataset được tải về tại: /root/.cache/kagglehub/datasets/hearfool/vggface2/versions/1


* 3.Kết nối vs Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


* lấy 93 thư mục

In [ ]:
import os
import shutil
import kagglehub

# Tải dataset VGGFace2 từ Kaggle
dataset_path = "/root/.cache/kagglehub/datasets/hearfool/vggface2/versions/1/train"

# Lấy danh sách tất cả thư mục con (danh tính)
all_folders = [f for f in os.listdir(dataset_path) if os.path.isdir(os.path.join(dataset_path, f))]

# Sắp xếp thư mục theo tên
all_folders.sort()

selected_folders = all_folders[0:93]


for folder in all_folders:
    if folder not in selected_folders:
        shutil.rmtree(os.path.join(dataset_path, folder))


#Khai báo

In [ ]:
import os
import cv2
import torch
import numpy as np
from PIL import Image
from mtcnn import MTCNN
from tqdm import tqdm
from facenet_pytorch import MTCNN
from google.colab.patches import cv2_imshow

In [ ]:
pip install facenet-pytorch

In [ ]:
pip  install mtcnn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 54.5 MB/s eta 0:00:00


#2.Tiền Xử lí

In [ ]:


# Định nghĩa đường dẫn dữ liệu
input_folder = "/root/.cache/kagglehub/datasets/hearfool/vggface2/versions/1/train"
output_folder = "/content/drive/My Drive/Processed_Images"
os.makedirs(output_folder, exist_ok=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Khởi tạo MTCNN để phát hiện khuôn mặt
detector = MTCNN(keep_all=False, device=device)



In [ ]:
def process_image(image_path, output_path):
    img = cv2.imread(image_path)

    # Kiểm tra xem ảnh có được đọc thành công hay không
    if img is None:
        return  # Bỏ qua ảnh nếu không đọc được

    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Chuyển ảnh sang RGB
    faces, _ = detector.detect(img_rgb, landmarks=False)

    # Kiểm tra nếu `faces` là None hoặc không có khuôn mặt nào
    if faces is None or len(faces) == 0:
        return



    x, y, w, h = map(int, faces[0])  # Chuyển tọa độ về số nguyên
    h_img, w_img, _ = img_rgb.shape  # Lấy kích thước ảnh
    if x < 0 or y < 0 or x + w > w_img or y + h > h_img:
      return  # Bỏ qua ảnh lỗi

    face = img_rgb[y:y+h, x:x+w]  # Cắt ảnh khuôn mặt từ ảnh gốc
    face_resized = cv2.resize(face, (160, 160))  # Resize về 112x112
    # print(f"đã lưu:{image_path}"
    Image.fromarray(face_resized).save(output_path)  # Lưu ảnh đã xử lý
    # print(f"đã lưu:{image_path}")


# Lặp qua từng ảnh trong dataset
for person in tqdm(os.listdir(input_folder)):
    person_folder = os.path.join(input_folder, person)
    output_person_folder = os.path.join(output_folder, person)
    os.makedirs(output_person_folder, exist_ok=True)

    for img_name in os.listdir(person_folder):
        img_path = os.path.join(person_folder, img_name)
        output_path = os.path.join(output_person_folder, img_name)
        process_image(img_path, output_path)

In [ ]:
import os

def count_images(folder_path, extensions=['.jpg', '.png', '.jpeg']):
    count = 0
    for root, _, files in os.walk(folder_path):
        count += sum(1 for file in files if any(file.lower().endswith(ext) for ext in extensions))
    return count

# Thay đổi đường dẫn thành thư mục cần kiểm tra
folder_path = "/content/drive/My Drive/Processed_Images"
total_images = count_images(folder_path)

print(f"Tổng số ảnh trong thư mục: {total_images}")


Tổng số ảnh trong thư mục: 1860


In [ ]:
import os
import random

# Thay đường dẫn bằng đường dẫn thư mục chứa các thư mục con
main_dir = "/content/drive/My Drive/Processed_Images"

def reduce_images_in_folders(main_dir, max_images=25):
    for subdir in os.listdir(main_dir):
        subdir_path = os.path.join(main_dir, subdir)

        if os.path.isdir(subdir_path):  # Kiểm tra nếu là thư mục
            images = [f for f in os.listdir(subdir_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

            if len(images) > max_images:
                images_to_remove = random.sample(images, len(images) - max_images)  # Chọn ảnh để xóa

                for img in images_to_remove:
                    img_path = os.path.join(subdir_path, img)
                    os.remove(img_path)

                print(f"Đã giảm thư mục {subdir} còn {max_images} ảnh.")

reduce_images_in_folders(main_dir)


#3.Train

In [ ]:
pip install torch torchvision


In [ ]:
pip install torch torchvision facenet-pytorch


#Chia dataset

In [ ]:
import os
import shutil
import random

def split_dataset(original_dir, output_dir, train_ratio=0.8):
    # Tạo folder output
    train_dir = os.path.join(output_dir, 'train')
    val_dir = os.path.join(output_dir, 'val')
    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(val_dir, exist_ok=True)

    # Duyệt từng class (người)
    for person in os.listdir(original_dir):
        person_dir = os.path.join(original_dir, person)
        if not os.path.isdir(person_dir):
            continue

        images = os.listdir(person_dir)
        random.shuffle(images)
        split_idx = int(len(images) * train_ratio)

        train_images = images[:split_idx]
        val_images = images[split_idx:]

        # Tạo thư mục class trong train/val
        os.makedirs(os.path.join(train_dir, person), exist_ok=True)
        os.makedirs(os.path.join(val_dir, person), exist_ok=True)

        # Copy ảnh vào thư mục mới
        for img in train_images:
            src = os.path.join(person_dir, img)
            dst = os.path.join(train_dir, person, img)
            shutil.copy2(src, dst)

        for img in val_images:
            src = os.path.join(person_dir, img)
            dst = os.path.join(val_dir, person, img)
            shutil.copy2(src, dst)

    print("✅ Dataset đã được chia thành train/val.")

# Gọi hàm
original_dataset = "/content/drive/My Drive/Processed_Images"
output_dataset = "/content/drive/My Drive/Split_Images"
split_dataset(original_dataset, output_dataset, train_ratio=0.8)


✅ Dataset đã được chia thành train/val.


#Định nghĩa trip

In [ ]:
from PIL import Image
from torch.utils.data import Dataset

class TripletFaceDataset(Dataset):
    def __init__(self, root_dir, transform):
        self.transform = transform
        self.classes = os.listdir(root_dir)
        self.class_to_images = {}

        for cls in self.classes:
            cls_path = os.path.join(root_dir, cls)
            if not os.path.isdir(cls_path): continue

            images = [os.path.join(cls_path, img) for img in os.listdir(cls_path)]
            if len(images) >= 2:
                self.class_to_images[cls] = images

        self.classes = list(self.class_to_images.keys())

    def __len__(self):
        return 10000  # Số lượng sample lấy ngẫu nhiên

    def __getitem__(self, idx):
        import random
        anchor_class = random.choice(self.classes)
        positive_class = anchor_class
        negative_class = random.choice([c for c in self.classes if c != anchor_class])

        anchor_path, positive_path = random.sample(self.class_to_images[anchor_class], 2)
        negative_path = random.choice(self.class_to_images[negative_class])

        anchor = self.transform(Image.open(anchor_path).convert("RGB"))
        positive = self.transform(Image.open(positive_path).convert("RGB"))
        negative = self.transform(Image.open(negative_path).convert("RGB"))

        return anchor, positive, negative


#load data

In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import DataLoader

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Transform
transform = transforms.Compose([
    transforms.Resize((112, 112)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

# Load train và val
train_dataset = TripletFaceDataset("/content/drive/My Drive/Split_Images/train", transform)
val_dataset = TripletFaceDataset("/content/drive/My Drive/Split_Images/val", transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Model
model = models.resnet50(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 512)
model = model.to(device)

# Loss và Optimizer
criterion = nn.TripletMarginLoss(margin=1.0, p=2)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


#train

In [ ]:
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for anchor, positive, negative in train_loader:
        anchor, positive, negative = anchor.to(device), positive.to(device), negative.to(device)

        optimizer.zero_grad()
        anchor_out = model(anchor)
        positive_out = model(positive)
        negative_out = model(negative)

        loss = criterion(anchor_out, positive_out, negative_out)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {total_loss:.4f}")


Epoch [1/10], Train Loss: 108.5397
Epoch [2/10], Train Loss: 44.2910
Epoch [3/10], Train Loss: 29.8469
Epoch [4/10], Train Loss: 24.5080
Epoch [5/10], Train Loss: 20.4178
Epoch [6/10], Train Loss: 14.2874
Epoch [7/10], Train Loss: 12.8980
Epoch [8/10], Train Loss: 15.9484
Epoch [9/10], Train Loss: 15.8148
Epoch [10/10], Train Loss: 15.4021


#Lưu model

In [ ]:
torch.save(model, "face_model_feature_v3.pth")
print("✅ Đã lưu model trích xuất đặc trưng.")


✅ Đã lưu model trích xuất đặc trưng.


#Đánh giá

In [ ]:
import torch.nn.functional as F

def evaluate(model, val_loader):
    model.eval()
    total = 0
    correct = 0
    cosine_similarities_pos = []
    cosine_similarities_neg = []

    with torch.no_grad():
        for anchor, positive, negative in val_loader:
            anchor, positive, negative = anchor.to(device), positive.to(device), negative.to(device)

            anchor_out = model(anchor)
            positive_out = model(positive)
            negative_out = model(negative)

            # Cosine similarity
            sim_pos = F.cosine_similarity(anchor_out, positive_out)
            sim_neg = F.cosine_similarity(anchor_out, negative_out)

            # Đánh giá: nếu sim_pos > sim_neg => đúng
            correct += torch.sum(sim_pos > sim_neg).item()
            total += anchor.size(0)

            cosine_similarities_pos.extend(sim_pos.cpu().numpy())
            cosine_similarities_neg.extend(sim_neg.cpu().numpy())

    accuracy = correct / total * 100
    print(f"✅ Độ chính xác trên val set: {accuracy:.2f}%")
    return cosine_similarities_pos, cosine_similarities_neg


In [ ]:
# Load lại model nếu cần
model = torch.load("face_model_feature_v3.pth", weights_only=False)
model.eval()

model.eval()

# Đánh giá trên val_loader
cosine_pos, cosine_neg = evaluate(model, val_loader)


✅ Độ chính xác trên val set: 90.50%
